In [ ]:
from IPython.display import display
import ipywidgets as ipw
import widgets
import utils
import os
import json

In [ ]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.get_aiidalab_eln_config()
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])

molecule_selector_config = {
    "dropdown": {"width": "315px"},
    "details": {"width": "415px", "height": "250px"},
    "image": {"width": "220px", "height": "250px"}
}
molecule_selection_widget = widgets.ObjectSelectionWidget("Molecule", molecule_selector_config)
molecule_selection_widget.load_dropdown_box("MOLECULE", "molecule")

instrument_selector = widgets.ObjectSelectionWidget("Instrument")
instrument_selector.load_dropdown_box("INSTRUMENT", "instrument")

srd_widgets = widgets.ObjectPropertiesWidgets("Sublimation Rate Determination")
srd_support_files = ipw.FileUpload(multiple = True)

increase_buttons_size = utils.HTML(data = ''.join(CONFIG["save_home_buttons_settings"]))
create_button = utils.Button(
    description = '', disabled = False, button_style = '', tooltip = 'Save', 
    icon = 'save', layout = ipw.Layout(width = '100px', height = '50px')
)
quit_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Main menu', icon = 'home', layout = ipw.Layout(width = '100px', height = '50px')
)
save_close_buttons_hbox = ipw.HBox([create_button, quit_button])

In [ ]:
def close_notebook(b):
    display(utils.Javascript(data = 'window.location.replace("home.ipynb")'))

def create_srd_openbis(b):
    object_properties = {}
    for prop in CONFIG["objects"]["SRD"]["properties"]:
        property_widget = srd_widgets.properties_widgets[prop]
        property_type = CONFIG["properties"][prop]["property_type"]
        object_properties[prop] = utils.get_object_property(property_type, property_widget)
        
    srd_parents = [molecule_selection_widget.dropdown.value, instrument_selector.dropdown.value]

    srd_object = utils.create_openbis_object(
        OPENBIS_SESSION, type="SUBLIMATION_RATE_DETERMINATION", 
        collection = "/LOG_ENTRIES/LOG_ENTRIES/SRD_COLLECTION", 
        props = object_properties, parents = srd_parents
    )
    utils.upload_datasets(OPENBIS_SESSION, srd_object, srd_support_files, "RAW_DATA")
    
    display(utils.Javascript(data = "alert('Upload successful!')"))

# Create sublimation rate determination object

## Select molecule

In [ ]:
display(molecule_selection_widget)

## Select instrument

In [ ]:
display(instrument_selector)

## Properties

In [ ]:
display(srd_widgets)

## Support files

In [ ]:
display(srd_support_files)

## Save

In [ ]:
molecule_selection_widget.dropdown.observe(molecule_selection_widget.load_metadata, names = 'value')
display(save_close_buttons_hbox)
display(increase_buttons_size)
create_button.on_click(create_srd_openbis)
quit_button.on_click(close_notebook)